In [ ]:
import torch
from transformers import ViTForImageClassification, ViTFeatureExtractor, ViTImageProcessor
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from PIL import Image
import os
import shutil
from sklearn.metrics import accuracy_score

In [2]:
# Custom Dataset Class
class DeepfakeDataset(Dataset):
    def __init__(self, image_dir, labels, transform):
        self.image_dir = image_dir
        self.labels = labels
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        image = Image.open(img_path).convert("RGB")
        label = self.labels[self.images[idx]]
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
# Load the pre-trained Google Vision Transformer
PRETRAINED_MODEL = "google/vit-base-patch16-224"

model = ViTForImageClassification.from_pretrained(PRETRAINED_MODEL)
processor = ViTImageProcessor.from_pretrained(PRETRAINED_MODEL)

In [15]:
# Prepare the Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

IMAGE_PATH = "./private_samples/"
AUTHENTIC = IMAGE_PATH + "authentic"
TAMPERED = IMAGE_PATH + "tampered"
TEMP_IMAGE_FOLDER = "./sample_images"

labels = {}
if os.path.isdir(TEMP_IMAGE_FOLDER):
    shutil.rmtree(TEMP_IMAGE_FOLDER)
os.mkdir(TEMP_IMAGE_FOLDER)

for f in os.listdir(AUTHENTIC):
    labels[f] = 0
    shutil.copy(AUTHENTIC + "/" + f, TEMP_IMAGE_FOLDER)

for f in os.listdir(TAMPERED):
    labels[f] = 1
    shutil.copy(TAMPERED + "/" + f, TEMP_IMAGE_FOLDER)

print(labels)
    
dataset = DeepfakeDataset(image_dir=TEMP_IMAGE_FOLDER, labels=labels, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

{'Au_txt_30028.jpg': 0, 'Au_txt_30026.jpg': 0, 'Au_txt_30027.jpg': 0, 'Au_txt_30025.jpg': 0, 'Au_txt_30024.jpg': 0, 'Tp_S_NRN_S_O_cha00077_cha00077_11017.jpg': 1, 'Tp_S_NRN_S_O_cha10187_cha10187_12308.jpg': 1, 'Tp_S_NRN_S_O_arc10129_arc10129_11895.jpg': 1, 'Tp_S_NRN_S_O_cha10126_cha10126_12153.jpg': 1, 'Tp_S_NRN_S_O_cha00035_cha00067_11734.jpg': 1}


In [18]:
optimizer = AdamW(model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss()

model.train()
for epoch in range(5):
    for images, labels in data_loader:
        inputs = processor(images, return_tensors="pt")
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")

Epoch 1: Loss = 0.7841014862060547
Epoch 2: Loss = 3.4363410472869873
Epoch 3: Loss = 1.3409761190414429
Epoch 4: Loss = 1.1505000591278076
Epoch 5: Loss = 0.981009840965271


In [ ]:
from sklearn.metrics import accuracy_score